In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter #chunks
from langchain_openai import OpenAIEmbeddings #embedding
from langchain_openai import ChatOpenAI #llm
from langchain_community.vectorstores import Chroma 
from langchain_community.document_loaders import PyPDFLoader 
from langchain.chains.question_answering import load_qa_chain


In [2]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = token

In [ ]:

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 200)

In [ ]:


pdf_link = "DOC-SF238339076816-20230503.pdf"

loader = PyPDFLoader(pdf_link, extract_images= False)
pages = loader.load_and_split()

In [ ]:
#Chunks

text_splitter = RecursiveCharacterTextSplitter (
    chunk_size = 400,
    chunk_overlap =20, 
    length_function = len,
    add_start_index = True
)
chunks = text_splitter.split_documents(pages)

In [ ]:
#Vector DB - Chroma

db =  Chroma.from_documents(chunks, embedding = embeddings_model, persist_directory= "text_index")
db.persist()

C:\Users\julia\AppData\Local\Temp\ipykernel_20892\3787942972.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [ ]:
#Load DB
vectordb = Chroma(persist_directory="text_index", embedding_function= embeddings_model)

#Load Retriever

retriever = vectordb.as_retriever(search_kwarfs= {"k":3})


chain =load_qa_chain(llm, chain_type="stuff")

C:\Users\julia\AppData\Local\Temp\ipykernel_20892\580150412.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="text_index", embedding_function= embeddings_model)
C:\Users\julia\AppData\Local\Temp\ipykernel_20892\580150412.py:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/m

In [15]:
def ask(question):
    context = retriever.get_relevant_documents(question) #seleciona o que considera mais importante
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
    return answer, context

In [16]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

User:  Quais os principais pontos da Lei que preciso ficar atenta na minha empresa?


C:\Users\julia\AppData\Local\Temp\ipykernel_20892\2049874228.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(question)
C:\Users\julia\AppData\Local\Temp\ipykernel_20892\2049874228.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']


Answer:  Os principais pontos da Lei que você precisa ficar atenta na sua empresa são:

1. Garantia contra a discriminação: É importante garantir que os sistemas de inteligência artificial não promovam discriminação.

2. Necessidade de revisão da inferência algorítmica: Deve haver um processo de revisão da lógica e funcionamento dos algoritmos utilizados.

3. Consulta e audiência públicas: Antes da implementação de regulamentos e normas, é necessário realizar consulta e audiência públicas.

4. Análise de impacto regulatório: Deve-se realizar análises de impacto regulatório antes de implementar regulamentos e normas.

5. Procedimentos para certificação do desenvolvimento e utilização de sistemas de alto risco: É importante seguir os procedimentos necessários para certificar o desenvolvimento e utilização de sistemas de alto risco.

Esses são


In [17]:
print(context)

[Document(metadata={'start_index': 1845, 'source': 'DOC-SF238339076816-20230503.pdf', 'page': 8}, page_content='proporcionais e estritamente necessárias ao atendimento do interesse público, \nobservados o devido processo legal e o controle judicial, bem como os \nprincípios e direitos previstos nesta Lei, especialmente a garantia contra a \ndiscriminação e a necessidade de revisão da inferência algorítmica pelo agente \nSF/23833.90768-16'), Document(metadata={'page': 21, 'start_index': 1675, 'source': 'DOC-SF238339076816-20230503.pdf'}, page_content='inteligência artificial qualificados como micro ou pequenas empresas, nos \ntermos da Lei Complementar nº 123, de 14 de dezembro de 2006, e startups, \nnos termos da Lei Complementar nº 182, de 1º de junho de 2021. \nArt. 33. A autoridade competente será o órgão central de \naplicação desta Lei e do estabelecimento de normas e diretrizes para a sua \nimplementação.'), Document(metadata={'page': 22, 'source': 'DOC-SF238339076816-20230503.pd